In [3]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime, timedelta
from tqdm import tqdm
import sys

In [2]:
naver_url = "https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=1"


In [4]:
#ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [27]:
original_html = requests.get(naver_url,headers=headers)
html = BeautifulSoup(original_html.text, "html.parser")

url_news = html.select_one("#main_content > div.list_body.newsflash_body > ul.type06_headline > li:nth-child(" + str(1) + ") > dl > dt:nth-child(2) > a")
url = url_news.attrs['href']

url

'https://n.news.naver.com/mnews/article/009/0005173068?sid=101'

In [10]:
b = "https://n.news.naver.com/mnews/article/421/0006991068?sid=101"

In [20]:
#각 기사 html get하기
news = requests.get(b)
news_html = BeautifulSoup(news.text,"html.parser")
# 뉴스 제목 가져오기
title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
if title == None:
    title = news_html.select_one("#content > div.end_ct > div > h2")

# 뉴스 본문 가져오기
content = news_html.select("#dic_area")
if content == []:
    content = news_html.select("#articeBody")

# 언론사 가져오기
try:
    press = news_html.select_one(".media_end_head_top_logo > img")['title']
except TypeError:
    press = news_html.select_one("#_CHANNEL_LAYER_366_0000868613 > span.media_end_head_top_channel_layer_text > strong")

# 언론사 코드 가져오기
#press_code = final_urls[39:41]

# 기사 텍스트만 가져오기
# list합치기
content = ''.join(str(content))

# html태그제거 및 텍스트 다듬기
pattern1 = '<[^>]*>'
title = re.sub(pattern=pattern1, repl='', string=str(title))
content = re.sub(pattern=pattern1, repl='', string=content)
press = re.sub(pattern=pattern1, repl='', string=str(press))
pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
content = content.replace(pattern2, '')

#날짜 가져오기

try:
    html_date = news_html.select_one("._ARTICLE_DATE_TIME")
    news_date = html_date.get_text('data-date-time')
except AttributeError:
    news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
    news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    
print("제목 : ",title)
print("본문 : ",content)
print("언론사 : ",press)
print("날짜 : ", news_date)

제목 :  '달러 강세'
본문 :  [




(서울=뉴스1) 김성진 기자 = 원·달러 환율이 지난 5월 18일(1334.2원) 이후 3개월 만에 1330원을 돌파했다. 미국의 7월 생산자물가지수(PPI)가 예상치를 웃돌면서 달러 강세를 보인데다 중국 경기둔화에 따른 위안화 약세가 원달러 환율을 끌어올리고 있다.사진은 15일 오전 서울 중구 명동 환전소 모습. 2023.8.15/뉴스1
		]
언론사 :  뉴스1
날짜 :  2023.08.15. 오후 2:00
